<a href="https://colab.research.google.com/github/viniciusrpb/datavis_book/blob/main/cap4_fundamentos_de_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [57]:
import nltk
from nltk.corpus import stopwords,reuters
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [59]:
nltk.download('punkt')
nltk.download('reuters')
nltk.download('stopwords')
stopwords = stopwords.words('english')

!unzip /root/nltk_data/corpora/reuters.zip -d /root/nltk_data/corpora/

Streaming output truncated to the last 5000 lines.
  inflating: /root/nltk_data/corpora/reuters/training/2231  
  inflating: /root/nltk_data/corpora/reuters/training/2232  
  inflating: /root/nltk_data/corpora/reuters/training/2234  
  inflating: /root/nltk_data/corpora/reuters/training/2236  
  inflating: /root/nltk_data/corpora/reuters/training/2237  
  inflating: /root/nltk_data/corpora/reuters/training/2238  
  inflating: /root/nltk_data/corpora/reuters/training/2239  
  inflating: /root/nltk_data/corpora/reuters/training/2240  
  inflating: /root/nltk_data/corpora/reuters/training/2244  
  inflating: /root/nltk_data/corpora/reuters/training/2246  
  inflating: /root/nltk_data/corpora/reuters/training/2247  
  inflating: /root/nltk_data/corpora/reuters/training/2249  
  inflating: /root/nltk_data/corpora/reuters/training/225  
  inflating: /root/nltk_data/corpora/reuters/training/2251  
  inflating: /root/nltk_data/corpora/reuters/training/2252  
  inflating: /root/nltk_data/corpor

In [24]:
documents = reuters.fileids()

train_docs_id = list(filter(lambda doc: doc.startswith("train"),documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"),documents))

train_docs = [reuters.raw(documento) for documento in train_docs_id]
test_docs = [reuters.raw(documento) for documento in test_docs_id]

In [25]:
#for x in train_docs:
#  print(x)

Streaming output truncated to the last 5000 lines.
  direction this country is going in."
      "That is the reason why the (Canadian) dollar today is
  higher than 75 cts (U.S.) compared to this time last year (when
  it was) a little over 69 cts," Wilson told the House of Commons
  daily question period.
      Figures released this week show foreigners purchased a
  record 23.1 billion dlrs of Canadian bonds in 1986, more than
  double the previous year, with Japan investing a record 9.5
  billion dlrs in the market.
      Wilson was responding to opposition party questions about a
  possible loss of jobs from the rise in Canadian investment
  abroad. Canadian investment, including the buying of foreign
  companies, stocks and bonds, rose to 12.53 billion dlrs from
  6.19 billion dlrs in 1985.
      The minister said the flow of funds from abroad would
  generate many new jobs in Canada.
  


CHARTWELL GROUP LTD &lt;CTWL> 4th qtr net
  Shr nine cts vs three cts
      Net 549,000 vs 7

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [60]:
tfidf = TfidfVectorizer(stop_words=stopwords)

tfidf_training = tfidf.fit(train_docs)

tfidf_training = tfidf.transform(train_docs)
tfidf_test = tfidf.transform(test_docs)

mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform([reuters.categories(documento) for documento in train_docs_id])
y_test = mlb.fit_transform([reuters.categories(documento) for documento in test_docs_id])

oneclass_train = y_train[:,0]
oneclass_test = y_test[:,0]

In [61]:
#tfidf_test.shape

tfidf_training.shape

(7769, 26147)

In [62]:
print(oneclass_train)

[0 1 0 ... 0 0 0]


In [63]:
clfsvm = SVC(kernel="rbf",gamma='scale',C=1)

clfsvm.fit(tfidf_training,oneclass_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [64]:
y_classified = clfsvm.predict(tfidf_test)

In [65]:
print(classification_report(oneclass_test,y_classified))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      2300
           1       0.98      0.95      0.97       719

    accuracy                           0.98      3019
   macro avg       0.98      0.97      0.98      3019
weighted avg       0.98      0.98      0.98      3019

